In [8]:
from transformers import BertTokenizerFast
from torch.utils.data import Dataset
from datasets import load_dataset
import torch

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')
sentence = "This is an example sentence. The quick fox jumps over the lazy dog"
tokens = tokenizer.tokenize(sentence)
print(tokens)


C:\Users\hexag\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['This', 'is', 'an', 'example', 'sentence', '.', 'The', 'quick', 'f', '##ox', 'jump', '##s', 'over', 'the', 'la', '##zy', 'dog']


In [ ]:
mnli_dataset = load_dataset("multi_nli")

Generating validation_mismatched split: 100%|██████████| 9832/9832 [00:00<00:00, 318583.45 examples/s]


KeyError: "Invalid key: 0. Please first select a split. For example: `my_dataset_dictionary['train'][0]`. Available splits: ['train', 'validation_matched', 'validation_mismatched']"

In [6]:
print(mnli_dataset['train'][0])

{'promptID': 31193, 'pairID': '31193n', 'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.', 'premise_binary_parse': '( ( Conceptually ( cream skimming ) ) ( ( has ( ( ( two ( basic dimensions ) ) - ) ( ( product and ) geography ) ) ) . ) )', 'premise_parse': '(ROOT (S (NP (JJ Conceptually) (NN cream) (NN skimming)) (VP (VBZ has) (NP (NP (CD two) (JJ basic) (NNS dimensions)) (: -) (NP (NN product) (CC and) (NN geography)))) (. .)))', 'hypothesis': 'Product and geography are what make cream skimming work. ', 'hypothesis_binary_parse': '( ( ( Product and ) geography ) ( ( are ( what ( make ( cream ( skimming work ) ) ) ) ) . ) )', 'hypothesis_parse': '(ROOT (S (NP (NN Product) (CC and) (NN geography)) (VP (VBP are) (SBAR (WHNP (WP what)) (S (VP (VBP make) (NP (NP (NN cream)) (VP (VBG skimming) (NP (NN work)))))))) (. .)))', 'genre': 'government', 'label': 1}
